# Global sea level budget discussion

In [83]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
# package
import os
import numpy as np
import plotly
import plotly.graph_objs as go
# script
from ncdf_io import ncdf_io
from regression import regress
from time_convert import time_convert




In [105]:
aisfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Antarctica/slf.t60.s300.m.deg1.scaMB_decom_multi.IVANS_IJ05_IMBIE.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'
gisfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Greenland/slf.t60.s300.m.deg1.scaMB_decom_multi.GERUO_ICE5_COMP.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'
lwmfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Landwater/slf.f.t60.s300.m.deg1.scaCLM4.5bgc_gict_hydt_ind_decom_multi.GERUO_ICE5_COMP.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'
aodfile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/GAC/slf.t100.s0.m.deg1.rf.200204_201410.regress.regts.agmask.-66_66.nc'
glofile='/Users/joedhsu/Research/Rsync/Data_t/GRACE05_CSR/004_154/Global/slf.landf.t60.s300.m.deg1.scaCLM4.5bgc_gict_hydt_ind_decom_multi_MB_decom_multi.IVANS_IJ05_IMBIE_GERUO_ICE5_COMP.GAC.t100.ptcorr.rf.200204_201410.regress.regts.agmask.-66_66.nc'
agofile='/Users/joedhsu/Research/Rsync/Data/Argo_Scripps/RG_ArgoClim_StericH_ano_swTEOS10.200401_201612.regress.regts.-66_66.nc'
altfile='/Users/joedhsu/Research/Rsync/Data/Altimetry/AVISO/monthly_mean_dt_upd/ncfiles/data.process.2015.0217vers.CMEMS/dt_global_allsat_msla_h_giay.0.5deg.20to7.199301_201512.regress.regts.agmask.-66_66.nc'

files=[aisfile,gisfile,lwmfile,aodfile,glofile,agofile,altfile]
varname=['anoSLF_SumPacAtlInd','anoSLF_SumPacAtlInd','anoSLF_SumPacAtlInd',
         'anoSLF_SumPacAtlInd','anoSLF_SumPacAtlInd','anoSH_SumPacAtlInd',
         'anoSL_SumPacAtlInd']
varname2=['ais','gis','lwm','aod','totmass','steric','totsl']

In [123]:
import datetime
import pandas as pd


def tarray_month(itime,ftime):
    """
    Parameters:
        itime: list 
            in the order of [year,month]
        ftime: list 
            in the order of [year,month]
    Returns:
        time : pd.indexes.timeindex
            time index in the form of pandas time index array
            
    """
    itime=np.array(itime,dtype=int)
    ftime=np.array(ftime,dtype=int)
    ini = datetime.datetime(itime[0],itime[1],1)
    fin = datetime.datetime(ftime[0],ftime[1],1)
    nmonth=((ftime[0]-1)-itime[0])*12+(12-itime[1]+1)+ftime[1]
    # 1st each month will be the output date
    time = pd.date_range(ini, freq='MS', periods=nmonth)
    # change array from 1st to 15th in each month 
    dtime=datetime.timedelta(days=14)
    time=time+dtime
   
    return time




In [124]:
# Dealing one file at a time 
import xarray as xr

for i in range(len(files)):
    ncfile=ncdf_io(files[i])
    ds=ncfile.read_ncdf2xarray()
    
    #old time stamp
    yeardate=np.array(ds['yeardate'])
    tdict=time_convert(yeardate).year2year_mon()
    year=tdict['year']
    month=tdict['month']

    # new time stamp
    tstamp=tarray_month([2002,1],[2016,12])
    
    # finding the missing month and assign NaN in the ts
    if i == 0:
        gts=np.zeros([len(files),len(tstamp)])+np.float('nan')

    # creating the new xarray dataset 
    for ii in range(len(year)):
        tsig=datetime.datetime(year[ii],month[ii],15)
        ind=np.where(tstamp==tsig)[0]
        gts[i,ind]=np.round(np.array(ds[varname[i]])[ii]-np.array(ds[varname[i]])[0],decimals=1)
        
    if i == 0:   
        dict_gts={varname2[i]:(['time'],gts[i,:])}
    else:
        dict_gts[varname2[i]]=(['time'],gts[i,:])       
    
    ds_corr = xr.Dataset(dict_gts,coords={'time': tstamp})
    


# gplt_ais = go.Scatter(name='Antarctic ice sheet',
#                          x=yeardate_str,
#                          y=gais)
# gplt_gis = go.Scatter(name='Greenland ice sheet',
#                          x=yeardate_str,
#                          y=ggis)
# gplt_lwm = go.Scatter(name="Land water storage",
#                          x=yeardate_str,
#                          y=glwm)
# gplt_aod = go.Scatter(name="Atmosphere water vapor",
#                          x=yeardate_str,
#                          y=gaod)
# gplt_glo = go.Scatter(name="Sum",
#                          x=yeardate_str,
#                          y=gglo)
# layout = dict(
#             title = 'Global Mean Sea Level',
#             showlegend = True,
#             calendar = "gregorian",
#             xaxis= dict(title='Year'),
#             yaxis= dict(title='Global Mean Sea Level (mm)')
#              )
    
# data=[gplt_ais,gplt_gis,gplt_lwm,gplt_aod,gplt_glo]

# py.iplot(dict(data=data,layout=layout),filename='plot_gsl')
# # plotly.offline.plot(dict(data=data,layout=layout),filename='plot_gsl.html'\
# #                          ,include_plotlyjs=False,output_type ='div' )
# # plotly.offline.plot(dict(data=data,layout=layout),filename='./figures/plot_gsl.html')
    
    
    
    




#   ncfile dimensions 
----------------------------------
time 139


#   ncfile variables 
----------------------------------
anoSLF_Pac (139,)
    dim: [u'time']
anoSLF_Atl (139,)
    dim: [u'time']
anoSLF_Ind (139,)
    dim: [u'time']
anoSLF_SumPacAtlInd (139,)
    dim: [u'time']
anoSLF_global (139,)
    dim: [u'time']
yeardate (139,)
    dim: [u'time']


#   ncfile dimensions 
----------------------------------
time 139


#   ncfile variables 
----------------------------------
anoSLF_Pac (139,)
    dim: [u'time']
anoSLF_Atl (139,)
    dim: [u'time']
anoSLF_Ind (139,)
    dim: [u'time']
anoSLF_SumPacAtlInd (139,)
    dim: [u'time']
anoSLF_global (139,)
    dim: [u'time']
yeardate (139,)
    dim: [u'time']


#   ncfile dimensions 
----------------------------------
time 139


#   ncfile variables 
----------------------------------
anoSLF_Pac (139,)
    dim: [u'time']
anoSLF_Atl (139,)
    dim: [u'time']
anoSLF_Ind (139,)
    dim: [u'time']
anoSLF_SumPacAtlInd (139,)
    dim: [u'ti

In [125]:
ds_corr

<xarray.Dataset>
Dimensions:  (time: 180)
Coordinates:
  * time     (time) datetime64[ns] 2002-01-15 2002-02-15 2002-03-15 ...
Data variables:
    aod      (time) float64 nan nan nan 0.0 0.0 nan nan -1.8 -0.5 0.3 0.9 ...
    gis      (time) float64 nan nan nan 0.0 0.1 nan nan 0.6 0.7 0.6 0.5 0.2 ...
    ais      (time) float64 nan nan nan 0.0 -0.1 nan nan 0.9 0.4 0.7 1.0 0.4 ...
    totsl    (time) float64 32.0 31.3 32.4 31.4 30.4 31.7 36.2 38.0 40.1 ...
    lwm      (time) float64 nan nan nan 0.0 2.1 nan nan 17.0 19.6 20.3 17.0 ...
    steric   (time) float64 nan nan nan nan nan nan nan nan nan nan nan nan ...
    totmass  (time) float64 nan nan nan 0.0 2.2 nan nan 16.6 20.2 21.9 19.5 ...

In [116]:
test={'mon':month}
test['year']=year
test

{'mon': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  5,
         6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,
        11, 12,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,
         4,  5,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  5,  6,  7,  8,
         9, 10, 11, 12,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  1,
         2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  5,  6,
         7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,
        12,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4,
         5,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  5,  6,  7,  8,  9,
        10, 11, 12,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  1,  2,
         3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  5,  6,  7,
         8,  9, 10, 11, 12,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
         1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  5,
     

### Plotting the global mean sea level contribution from mass changes 

In [305]:


plotly.offline.init_notebook_mode(connected=True)

yeardate=np.array(ds_ais['yeardate'])
dict1=time_convert(yeardate).year2year_mon()
# dict1=year2year_mon(yeardate)
yeardate_str=dict1['string']
gais=np.round(np.array(ds_ais['anoSLF_global'])-np.array(ds_ais['anoSLF_global'])[0],decimals=1)
ggis=np.round(np.array(ds_gis['anoSLF_global'])-np.array(ds_gis['anoSLF_global'])[0],decimals=1)
glwm=np.round(np.array(ds_lwm['anoSLF_global'])-np.array(ds_lwm['anoSLF_global'])[0],decimals=1)
gaod=np.round(np.array(ds_aod['anoSLF_global'])-np.array(ds_aod['anoSLF_global'])[0],decimals=1)
gglo=np.round(np.array(ds_glo['anoSLF_global'])-np.array(ds_glo['anoSLF_global'])[0],decimals=1)


gplt_ais = go.Scatter(name='Antarctic ice sheet',
                         x=yeardate_str,
                         y=gais)
gplt_gis = go.Scatter(name='Greenland ice sheet',
                         x=yeardate_str,
                         y=ggis)
gplt_lwm = go.Scatter(name="Land water storage",
                         x=yeardate_str,
                         y=glwm)
gplt_aod = go.Scatter(name="Atmosphere water vapor",
                         x=yeardate_str,
                         y=gaod)
gplt_glo = go.Scatter(name="Sum",
                         x=yeardate_str,
                         y=gglo)
layout = dict(
            title = 'Global Mean Sea Level',
            showlegend = True,
            calendar = "gregorian",
            xaxis= dict(title='Year'),
            yaxis= dict(title='Global Mean Sea Level (mm)')
             )
    
data=[gplt_ais,gplt_gis,gplt_lwm,gplt_aod,gplt_glo]

py.iplot(dict(data=data,layout=layout),filename='plot_gsl')
# plotly.offline.plot(dict(data=data,layout=layout),filename='plot_gsl.html'\
#                          ,include_plotlyjs=False,output_type ='div' )
# plotly.offline.plot(dict(data=data,layout=layout),filename='./figures/plot_gsl.html')